# Комп'ютерний практикум

In [ ]:
import hashlib as hl    # sha1
import secrets as rnd   # randbelow, choice

In [ ]:
# Some hardcoded stuff specifically for sha1
text = ' \t\n\r\v\fabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
punct = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
ALPH = text + punct

PIA_CMP_PREF_C = 40 - 4    # 4 chars to compare in the end
BDA_CMP_PREF_C = 40 - 8    # 8 chars to compare in the end

PIA_CMP_PREF_B = 20 - 2    # 2 bytes to compare in the end
BDA_CMP_PREF_B = 20 - 4    # 4 bytes to compare in the end
# ------------------------------------------

BASE_MESSAGE_PARTS = ["Bondar", "Petro", "Olexandrovych", ""]

In [ ]:
def gen_bm_sample(char_counts=[0, 0, 0, 0]):
    result_msg_parts = []

    for i in range(4):
        if char_counts[i] != 0:
            gen_part = str()
            for _ in range(4):
                gen_part += rnd.choice(ALPH[6:])
            result_msg_parts.append(gen_part)
        result_msg_parts.append(BASE_MESSAGE_PARTS[i])

    return "".join(result_msg_parts)

def random_modification(base: str):
    res = list(base)

    for i in range(len(res)):
        if rnd.randbelow(2) == 1:
            res[i] = rnd.choice(ALPH)

    return "".join(res)


In [ ]:
def preimage_attack_1(base_message: str):
    print(f"Base message is: {base_message}\n")
    # Formating parameters
    pw = len(base_message) + 6
    #

    base_sha = hl.sha1(base_message.encode())
    print(f"{base_message:{pw}s} : {base_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{base_sha.hexdigest()[PIA_CMP_PREF_C:]}\n")

    app = 1
    while True:
        msg = base_message + str(app)
        msg_sha = hl.sha1(msg.encode())

        print(f"{msg:{pw}s} : {msg_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{msg_sha.hexdigest()[PIA_CMP_PREF_C:]}", end='')

        if msg_sha.digest()[PIA_CMP_PREF_B:] == base_sha.digest()[PIA_CMP_PREF_B:]:
            print(f"\tSecond preimage on {app}!")
            break
        else:
            app += 1
            print()

In [ ]:
def preimage_attack_2(base_message: str):
    print(f"Base message is: {base_message}\n")
    # Formating parameters
    pw = len(base_message) + 6
    #

    base_sha = hl.sha1(base_message.encode())
    print(f"{repr(base_message):{pw}s} : {base_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{base_sha.hexdigest()[PIA_CMP_PREF_C:]}\n")

    itr = 0
    while True:
        itr += 1
        msg = random_modification(base_message)
        msg_sha = hl.sha1(msg.encode())

        print(f"{repr(msg):{pw}s} : {msg_sha.hexdigest()[:PIA_CMP_PREF_C]}\t{msg_sha.hexdigest()[PIA_CMP_PREF_C:]}", end='')

        if msg_sha.digest()[PIA_CMP_PREF_B:] == base_sha.digest()[PIA_CMP_PREF_B:] and msg != base_message:
            print(f"\tSecond preimage in {itr} iterations!")
            break
        else:
            print()
            continue

In [ ]:
def birthday_attack_1(base_message: str):
    print(f"Base message is: {base_message}\n")
    # Formating parameters
    pw = len(base_message) + 6
    #

    base_sha = hl.sha1(base_message.encode())
    print(f"{repr(base_message):{pw}s} : {base_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{base_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")

    prev_dict = {base_sha.digest()[BDA_CMP_PREF_B:]: base_message}

    app = 1
    while True:
        msg = base_message + str(app)
        msg_sha = hl.sha1(msg.encode())

        if msg_sha.digest()[BDA_CMP_PREF_B:] in prev_dict.keys():
            col = prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]]
            print("\nCollision found!")
            print(f"Msg: {msg:{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}")
            col_sha = hl.sha1(col.encode())
            print(f"Col: {col:{pw}s} : {col_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{col_sha.hexdigest()[BDA_CMP_PREF_C:]}")
            break
        else:
            print(f"{repr(msg):{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}")
            prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]] = msg
            app += 1

In [ ]:
def birthday_attack_2(base_message: str):
    print(f"Base message is: {base_message}\n")
    # Formating parameters
    pw = len(base_message) + 6
    #

    base_sha = hl.sha1(base_message.encode())
    print(f"{repr(base_message):{pw}s} : {base_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{base_sha.hexdigest()[BDA_CMP_PREF_C:]}\n")

    prev_dict = {base_sha.digest()[BDA_CMP_PREF_B:]: base_message}

    itr = 0
    while True:
        itr += 1
        msg = random_modification(base_message)
        msg_sha = hl.sha1(msg.encode())

        if msg_sha.digest()[BDA_CMP_PREF_B:] in prev_dict.keys() and msg != base_message:
            col = prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]]
            print(f"\nCollision found in {itr} iterations!")
            print(f"Msg: {repr(msg):{pw}s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}")
            col_sha = hl.sha1(col.encode())
            print(f"Col: {repr(col):{pw}s} : {col_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{col_sha.hexdigest()[BDA_CMP_PREF_C:]}")
            break
        else:
            print(f"{repr(msg):40s} : {msg_sha.hexdigest()[:BDA_CMP_PREF_C]}\t{msg_sha.hexdigest()[BDA_CMP_PREF_C:]}")
            prev_dict[msg_sha.digest()[BDA_CMP_PREF_B:]] = msg

## Запуск атак

### Атака пошуку другого прообразу 1

In [ ]:
preimage_attack_1(gen_bm_sample([5, 0, 0, 0]))

### Атака пошуку другого прообразу 2

In [ ]:
preimage_attack_2(gen_bm_sample([0, 5, 0, 0]))

### Атака днів народжень 1

In [ ]:
birthday_attack_1(gen_bm_sample([0, 0, 5, 0]))

### Атака днів народжень 2

In [ ]:
birthday_attack_2(gen_bm_sample([0, 0, 0, 5]))